In [1]:
%load_ext autotime

In [2]:
import imp # need to be done for python 3
import logging
imp.reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', 
                    level=logging.DEBUG, datefmt='%I:%M:%S')

time: 3.87 ms


In [3]:
import sys
import os

time: 1.13 ms


In [4]:
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/magellan/')

time: 853 µs


In [5]:
import magellan as mg

time: 2.6 s


In [6]:
path_a = os.sep.join(['data', 'DBLP_demo.csv'])
path_b = os.sep.join(['data', 'ACM_demo.csv'])
path_c = os.sep.join(['data', 'dblp_acm_demo_labels.csv'])

time: 1.46 ms


In [7]:
A = mg.read_csv_metadata(path_a, key='id')
B = mg.read_csv_metadata(path_b, key='id')
C = mg.read_csv_metadata(path_c, key='_id', ltable=A, rtable=B, fk_ltable='ltable.id', 
                        fk_rtable='rtable.id')

11:08:32 WARNING:Metadata file is not present in the given path; proceeding to read the csv file.
11:08:32 WARNING:Metadata file is not present in the given path; proceeding to read the csv file.
11:08:32 WARNING:Metadata file is not present in the given path; proceeding to read the csv file.


time: 99.8 ms


In [8]:
feature_table = mg.get_features_for_matching(A, B)

11:08:32 WARNING:Magellan types: id type (str_eq_1w) and id type (numeric) are different.If you want to set them to be same and generate features, update output from get_attr_types and use get_features command.

11:08:32 WARNING:Magellan types: venue type (str_bt_1w_5w) and venue type (str_bt_5w_10w) are different.If you want to set them to be same and generate features, update output from get_attr_types and use get_features command.



time: 143 ms


In [9]:
feature_vectors = mg.extract_feature_vecs(C, feature_table=feature_table, attrs_after='gold', 
                                          verbose=True)

11:08:32 INFO:Required metadata: cand.set key, fk ltable, fk rtable, ltable, rtable, ltable key, rtable key
11:08:32 INFO:Getting metadata from the catalog
11:08:32 INFO:..... Done
11:08:32 INFO:Validating candset key: _id
11:08:32 INFO:..... Done
11:08:32 INFO:Validating foreign key constraint for left table
11:08:32 INFO:..... Done
11:08:32 INFO:Validating foreign key constraint for right table
11:08:32 INFO:..... Done


time: 1.29 s


In [19]:
help(mg.impute_table)

Help on function impute_table in module magellan.utils.generic_helper:

impute_table(table, exclude_attrs=None, missing_val='NaN', strategy='mean', axis=0, val_all_nans=0)
    Impute table
    Parameters
    ----------
    table : MTable, for which values should be imputed
    exclude_attrs : list of attribute names to be excluded from imputing.
    missing_val : String, specifies the missing value format.
    strategy : String, on how to impute values. Valid strings: 'mean', 'median', 'most_frequent'
    axis : int, 0/1. axis=1 along rows, and axis=0 along columns.
    val_all_nans: float. Value fto fill in if all the other values are NaN.
    Returns
    -------
    result : Imputed table.

time: 994 µs


In [10]:
feature_vectors.fillna(0, inplace=True)

time: 78.2 ms


In [11]:
train_test = mg.train_test_split(feature_vectors)

11:08:34 INFO:Required metadata: cand.set key, fk ltable, fk rtable, ltable, rtable, ltable key, rtable key
11:08:34 INFO:Getting metadata from the catalog
11:08:34 INFO:..... Done
11:08:34 INFO:Validating candset key: _id
11:08:34 INFO:..... Done
11:08:34 INFO:Validating foreign key constraint for left table
11:08:34 INFO:..... Done
11:08:34 INFO:Validating foreign key constraint for right table
11:08:34 INFO:..... Done


time: 31.8 ms


In [12]:
train, test = train_test['train'], train_test['test']

time: 1.36 ms


In [13]:
dt = mg.DTMatcher()

time: 1.11 ms


In [14]:
dt.fit(table=train, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
               target_attr='gold')

time: 5.76 ms


In [15]:
help(dt.predict)

Help on method predict in module magellan.matcher.mlmatcher:

predict(x=None, table=None, exclude_attrs=None, target_attr=None, append=False, inplace=True) method of magellan.matcher.dtmatcher.DTMatcher instance
    Predict interface for the matchers.
    
    Specifically, there are two ways the user can call the predict method.
    First, interface similar to scikit-learn where the feature vectors
    given as projected DataFrame.
    Second, give the DataFrame and explicitly specify the feature vectors
    (by specifying the attributes to be excluded) .
    
    A point to note is all the input parameters have a default value of
    None. This is done to support both the interfaces in a single function.
    
    
    Args:
        x (DataFrame): Input pandas DataFrame containing only feature
            vectors (defaults to None).
        table (DataFrame): Input pandas DataFrame containing feature
            vectors, and may be other attributes (defaults to None).
        exclude_

In [16]:
dt.predict(table=test, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], 
           target_attr='predicted',
            append=True, inplace=True)

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,authors_authors_jac_qgm_3_qgm_3,authors_authors_cos_dlm_dc0_dlm_dc0,authors_authors_mel,authors_authors_lev_dist,authors_authors_lev_sim,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,gold,predicted
296,296,journals/sigmod/Group94,187439,0.684211,0.716115,0.896591,14.0,0.831325,0.695652,0.894427,0.785218,10.0,0.761905,1.0,1.000000,2.0,0.666667,1,1
338,338,journals/sigmod/RossHLW01,764214,0.016393,0.000000,0.571503,28.0,0.125000,0.072727,0.204124,0.638662,51.0,0.281690,0.0,0.999500,2.0,0.666667,0,0
60,60,conf/sigmod/GibbsBT94,191856,0.489362,0.400000,0.906688,4.0,0.878788,0.915254,0.925820,0.924659,3.0,0.947368,1.0,1.000000,2.0,0.666667,1,1
227,227,conf/vldb/RothOH99,671526,0.024390,0.000000,0.504663,79.0,0.177083,0.038462,0.000000,0.414683,37.0,0.119048,1.0,1.000000,2.0,0.666667,0,0
24,24,conf/sigmod/BunemanDHS96,373697,0.191489,0.272166,0.604299,51.0,0.227273,0.150000,0.223607,0.594333,46.0,0.269841,0.0,0.997501,6.0,0.000000,0,0
256,256,conf/vldb/ZirintsisKM99,872855,0.000000,0.000000,0.500389,31.0,0.205128,0.056122,0.138675,0.573903,147.0,0.169492,0.0,0.998003,6.0,0.000000,0,0
151,151,conf/vldb/AnciauxBP03,223791,0.130435,0.111111,0.621037,53.0,0.231884,0.220930,0.333333,0.684748,27.0,0.490566,0.0,0.996006,6.0,0.000000,0,0
137,137,conf/sigmod/ZhuS03,872780,0.605263,0.444444,0.908285,5.0,0.918033,0.542857,0.500000,0.577778,20.0,0.200000,1.0,1.000000,2.0,0.666667,1,1
286,286,journals/sigmod/DogacDKOONEHAKKM95,212017,0.600000,0.250000,0.958254,3.0,0.911765,0.437158,0.426401,0.686485,122.0,0.273810,1.0,1.000000,2.0,0.666667,1,1
330,330,journals/sigmod/PitouraB95,765538,0.164384,0.188982,0.639683,67.0,0.379630,0.028571,0.200000,0.583850,31.0,0.162162,0.0,0.998498,3.0,0.500000,0,0


time: 234 ms


In [17]:
result = mg.eval_matches(test, 'gold', 'predicted')

time: 21.7 ms


In [18]:
mg.print_eval_summary(result)

Precision : 98.21% (110/112)
Recall : 97.35% (110/113)
F1 : 97.78%
False positives : 2 (out of 112 positive predictions)
False negatives : 3 (out of 96 negative predictions)
time: 1.25 ms
